## Analyse FF Mouse Brain Coronal Section with SpaGCN

In [35]:
import os,csv,re
import pandas as pd
import numpy as np
import numba
import scanpy as sc
import math
import SpaGCN as spg
from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import SpaGCN as spg
import cv2

In [36]:
##Read in preprocessed adata
#Read in gene expression and spatial location
adata=sc.read("ff_mouse_brain_coronal.h5ad")

#read in spatial positions
spatial=pd.read_csv("spatial/tissue_positions_list.csv",sep=",",header=None,na_filter=False,index_col=0,names=["barcode", "in_tissue", "array_row", "array_col","pxl_row_in_fullres","pxl_col_in_fullres"]) 

#Read in hitology image
img=cv2.imread("spatial/tissue_hires_image.tiff")


In [37]:
##re index spatial table according to barcodes in adata
sorted_barcodes = list(adata.obs_names)

spatial = spatial.reindex(sorted_barcodes)

In [38]:
# Manually edit the 'log1p' values
adata.uns['log1p'] = {'base': None}


In [39]:
#Set coordinates
x_array=adata.obs["array_row"].tolist()
y_array=adata.obs["array_col"].tolist()
x_pixel=spatial["pxl_row_in_fullres"].tolist()
y_pixel=spatial["pxl_col_in_fullres"].tolist()

print(img)
#Test coordinates on the image
img_new=img.copy()
for i in range(len(x_pixel)):
    x=x_pixel[i]
    y=y_pixel[i]
    img[int(x-20):int(x+20), int(y-20):int(y+20),:]=0

cv2.imwrite('ff_mouse_brain_coronal_section_scanpy_processed_image_map.jpg', img_new)

[[[125 132 129]
  [125 131 129]
  [127 131 129]
  ...
  [125 130 127]
  [126 129 127]
  [125 129 127]]

 [[126 131 129]
  [125 131 129]
  [126 131 129]
  ...
  [123 129 126]
  [126 129 127]
  [127 130 127]]

 [[125 131 129]
  [127 131 129]
  [127 131 128]
  ...
  [125 130 126]
  [126 129 126]
  [126 131 128]]

 ...

 [[126 132 129]
  [126 132 129]
  [127 132 129]
  ...
  [127 132 128]
  [126 131 128]
  [126 132 129]]

 [[128 131 129]
  [127 132 129]
  [126 132 128]
  ...
  [126 131 128]
  [126 131 129]
  [126 131 130]]

 [[127 132 130]
  [127 132 130]
  [127 131 129]
  ...
  [126 131 128]
  [125 132 129]
  [127 131 129]]]


True

In [40]:
#Calculate adjacent matrix
s=1
b=49
#If histlogy image is not available, SpaGCN can calculate the adjacent matrix using the fnction below
adj=spg.calculate_adj_matrix(x=x_pixel,y=y_pixel, histology=False)
print(adj)
np.savetxt('ff_mouse_brain_coronal_section_adj.csv', adj, delimiter=',')

Calculateing adj matrix using xy only...
[[   0.      6937.9736  5811.85    ... 5096.56    5195.5293  6230.7305 ]
 [6937.9736     0.      6904.304   ... 6881.32    5147.134    728.68646]
 [5811.85    6904.304      0.      ...  766.29626 1763.9376  6603.5957 ]
 ...
 [5096.56    6881.32     766.29626 ...    0.      1806.8962  6507.2393 ]
 [5195.5293  5147.134   1763.9376  ... 1806.8962     0.      4840.279  ]
 [6230.7305   728.68646 6603.5957  ... 6507.2393  4840.279      0.     ]]


In [41]:
#set hyper-parameters
p=0.5 
#Find the l value given p
l=spg.search_l(p, adj, start=0.01, end=1000, tol=0.01, max_run=100)
print(l)

Run 1: l [0.01, 1000], p [0.0, 277.83445245317006]
Run 2: l [0.01, 500.005], p [0.0, 79.11184692382812]
Run 3: l [0.01, 250.0075], p [0.0, 20.416732788085938]
Run 4: l [0.01, 125.00874999999999], p [0.0, 4.566183567047119]
recommended l =  62.509375
62.509375


In [42]:
#Going with 9 clusters as this is what I got from previous data - this has 0 index
n_clusters=18
#Set seed
r_seed=t_seed=n_seed=100
#Search for suitable resolution
res=spg.search_res(adata, adj, l, n_clusters, start=0.7, step=0.1, tol=5e-3, lr=0.05, max_epochs=20, r_seed=r_seed, t_seed=t_seed, n_seed=n_seed)

Start at res =  0.7 step =  0.1
Initializing cluster centers with louvain, resolution =  0.7
Epoch  0
Epoch  10
Res =  0.7 Num of clusters =  14
Initializing cluster centers with louvain, resolution =  0.7999999999999999
Epoch  0
Epoch  10
Res =  0.7999999999999999 Num of clusters =  15
Res changed to 0.7999999999999999
Initializing cluster centers with louvain, resolution =  0.8999999999999999
Epoch  0
Epoch  10
Res =  0.8999999999999999 Num of clusters =  18
recommended res =  0.8999999999999999


In [43]:
clf=spg.SpaGCN()
clf.set_l(l)
#Set seed
random.seed(r_seed)
torch.manual_seed(t_seed)
np.random.seed(n_seed)
#Run
clf.train(adata,adj,init_spa=True,init="louvain",res=res, tol=5e-3, lr=0.05, max_epochs=200)
y_pred, prob=clf.predict()
adata.obs["pred"]= y_pred
adata.obs["pred"]=adata.obs["pred"].astype('category')
#Do cluster refinement(optional)
#shape="hexagon" for Visium data, "square" for ST data.
adj_2d=spg.calculate_adj_matrix(x=x_array,y=y_array, histology=False)
refined_pred=spg.refine(sample_id=adata.obs.index.tolist(), pred=adata.obs["pred"].tolist(), dis=adj_2d, shape="hexagon")
adata.obs["refined_pred"]=refined_pred
adata.obs["refined_pred"]=adata.obs["refined_pred"].astype('category')

Initializing cluster centers with louvain, resolution =  0.8999999999999999
Epoch  0
Epoch  10
Epoch  20
Epoch  30
Epoch  40
delta_label  0.003581376840429765 < tol  0.005
Reach tolerance threshold. Stopping training.
Total epoch: 40
Calculateing adj matrix using xy only...


In [44]:
#Plot spatial domains
#Set colors used
plot_color=["#F56867","#FEB915","#C798EE","#59BE86","#7495D3","#D1D1D1","#6D1A9C","#15821E","#3A84E6","#997273","#787878","#DB4C6C","#9E7A7A","#554236","#AF5F3C","#93796C","#F9BD3F","#DAB370","#877F6C","#268785"]
#Plot spatial domains
domains="pred"
num_celltype=len(adata.obs[domains].unique())
adata.uns[domains+"_colors"]=list(plot_color[:num_celltype])
#ax=sc.pl.scatter(adata,alpha=1,x="y_pixel",y="x_pixel",color=domains,title=domains,color_map=plot_color,show=False,size=100000/adata.shape[0])
#change above line so it comes from adata input
ax=sc.pl.scatter(adata,alpha=1,x="array_row",y="array_col",color=domains,title=domains,color_map=plot_color,show=False,size=100000/adata.shape[0])
ax.set_aspect('equal', 'box')
ax.axes.invert_yaxis()
plt.savefig("ff_mouse_brain_coronal_section_spagcn_pred.png", dpi=600)
plt.close()

#Plot refined spatial domains
domains="refined_pred"
num_celltype=len(adata.obs[domains].unique())
adata.uns[domains+"_colors"]=list(plot_color[:num_celltype])
ax=sc.pl.scatter(adata,alpha=1,x="array_row",y="array_col",color=domains,title=domains,color_map=plot_color,show=False,size=100000/adata.shape[0])
ax.set_aspect('equal', 'box')
ax.axes.invert_yaxis()
plt.savefig("ff_mouse_brain_coronal_section_spagcn_refined_pred.png", dpi=600)
plt.close()

In [45]:
#Use domain 0 as an example
target=0
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 14.142135620117188], num_nbr [1.0, 265.6431924882629]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 7.778174579143524], num_nbr [1.0, 89.49295774647888]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 4.5961940586566925], num_nbr [1.0, 35.056338028169016]
Calculateing adj matrix using xy only...
recommended radius =  3.0052037984132767 num_nbr=12.47887323943662
3.0052037984132767
radius= 3.0052037984132767 average number of neighbors for each spot is 12.47887323943662
 Cluster 0 has neighbors:
Dmain  6 :  215
Dmain  17 :  200
Dmain  9 :  179
Dmain  7 :  95
Dmain  8 :  24
[6, 17, 9, 7, 8]
SVGs for domain  0 : ['Hcn2', 'Nefl', 'Sncb', 'Cplx1', 'Rnf227', 'Oip5os1', 'Rph3a', 'Nat8l', 'Hpca', 'Vamp1', 'Pcp4l1', 'Gad2', 'Kcnc1', 'Fndc5', 'Cygb', 'Nefm', 'Gad1']


In [47]:
target=1
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 14.142135620117188], num_nbr [1.0, 253.1038961038961]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 7.778174579143524], num_nbr [1.0, 77.26406926406926]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 4.5961940586566925], num_nbr [1.0, 28.92207792207792]
Calculateing adj matrix using xy only...
recommended radius =  3.0052037984132767 num_nbr=10.225108225108226
3.0052037984132767
radius= 3.0052037984132767 average number of neighbors for each spot is 10.225108225108226
 Cluster 1 has neighbors:
Dmain  11 :  244
[11]
SVGs for domain  1 : ['Camk2a', 'Atp1a3', 'Dnm1', 'Olfm1', 'Epn1', 'Pfn2', 'Cox7b', 'Ywhag', 'Gabarapl1', 'Rtn1', 'Ndrg4', 'Uchl1', 'Stmn3', 'Map1b', 'Ywhab', 'Map1a', 'Cnih2', 'Snap25', 'Pgam1', 'Ly6h', 'Map2', 'Thy1', 'Nptn', 'Nsmf', 'Ddn', 'Atp1b1', 'Ncdn', '

In [48]:
target=2
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 14.142135620117188], num_nbr [1.0, 224.97802197802199]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 7.778174579143524], num_nbr [1.0, 85.56043956043956]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 4.5961940586566925], num_nbr [1.0, 34.146520146520146]
Calculateing adj matrix using xy only...
recommended radius =  3.0052037984132767 num_nbr=12.267399267399268
3.0052037984132767
radius= 3.0052037984132767 average number of neighbors for each spot is 12.267399267399268
 Cluster 2 has neighbors:
Dmain  3 :  353
Dmain  5 :  338
Dmain  14 :  48
[3, 5, 14]
SVGs for domain  2 : ['Nrsn1', 'Cplx1', 'Rgs4', 'Kcna2', 'Lynx1', 'Sorl1', 'Epha4', 'Satb1', 'Ccdc136', 'Pvalb', 'Plcxd2', 'Osbp2', 'Cnih3', 'Nrep', 'Dkkl1']


In [49]:
target=3
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 14.142135620117188], num_nbr [1.0, 178.0]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 7.778174579143524], num_nbr [1.0, 66.33488372093024]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 4.5961940586566925], num_nbr [1.0, 28.837209302325583]
Calculateing adj matrix using xy only...
recommended radius =  3.0052037984132767 num_nbr=11.144186046511628
3.0052037984132767
radius= 3.0052037984132767 average number of neighbors for each spot is 11.144186046511628
 Cluster 3 has neighbors:
Dmain  2 :  353
Dmain  4 :  55
[2, 4]
SVGs for domain  3 : ['Cplx2', 'Ly6h', 'Ddn', 'Nptxr', 'Hpcal4', 'Itpka', 'Rasgef1a', 'Ppp1r1a', 'Enpp2', 'Lamp5', 'Calb1', '2010300C02Rik', 'Atp2b4', 'Npy', 'Igfbp6', 'Ccn3', 'Rasgrf2', 'Tesc', 'Pitpnm2', 'Kctd1', 'Cux2', 'Trp53i11', 'Dgkb', 'Sowaha', 'S10

In [50]:
target=4
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 14.142135620117188], num_nbr [1.0, 141.0]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 7.778174579143524], num_nbr [1.0, 47.785714285714285]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 4.5961940586566925], num_nbr [1.0, 19.642857142857142]
Calculateing adj matrix using xy only...
Run 4: radius [3.0052037984132767, 4.5961940586566925], num_nbr [7.571428571428571, 19.642857142857142]
Calculateing adj matrix using xy only...
recommended radius =  3.8006989285349846 num_nbr=12.0
3.8006989285349846
radius= 3.8006989285349846 average number of neighbors for each spot is 12.0
 Cluster 4 has neighbors:
No neighbor domain found, try bigger radius or smaller ratio.
None


TypeError: 'NoneType' object is not subscriptable

In [51]:
target=5
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 14.142135620117188], num_nbr [1.0, 268.6878306878307]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 7.778174579143524], num_nbr [1.0, 92.96296296296296]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 4.5961940586566925], num_nbr [1.0, 36.026455026455025]
Calculateing adj matrix using xy only...
recommended radius =  3.0052037984132767 num_nbr=12.751322751322752
3.0052037984132767
radius= 3.0052037984132767 average number of neighbors for each spot is 12.751322751322752
 Cluster 5 has neighbors:
Dmain  2 :  338
Dmain  11 :  225
Dmain  14 :  64
[2, 11, 14]
SVGs for domain  5 : ['Ttc9b', 'Khdrbs3', 'Uchl1', 'Atp2b1', 'Ngef', 'Tmem132a', 'Tmsb10', 'Tuba4a', 'Basp1', 'Hpcal4', 'Ndrg3', 'Ctxn1', 'Slc17a7', 'Id2', 'Pcp4', 'Diras2', 'Efhd2', 'Mllt11', 'Vsnl1', 'Ncald', 'Dkk3', 'Ra

In [52]:
target=6
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 14.142135620117188], num_nbr [1.0, 288.4122807017544]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 7.778174579143524], num_nbr [1.0, 94.53508771929825]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 4.5961940586566925], num_nbr [1.0, 36.421052631578945]
Calculateing adj matrix using xy only...
recommended radius =  3.0052037984132767 num_nbr=12.87719298245614
3.0052037984132767
radius= 3.0052037984132767 average number of neighbors for each spot is 12.87719298245614
 Cluster 6 has neighbors:
Dmain  0 :  215
Dmain  11 :  96
Dmain  8 :  28
[0, 11, 8]
SVGs for domain  6 : ['Trf', 'Sez6l2', 'Tmbim6', 'Rhob', 'Ptprd', 'Cers2', 'Prdx1', 'Golga7', 'Elavl3', 'Mal', 'Mog', 'Kif1b', 'Lamp1', 'Sirt2', 'Tmem151a', 'Abca2', 'Scd1', 'Enpp2', 'Pcbp4', 'Npc2', 'Fnbp1', 'Tppp', 'Selenop',

In [53]:
target=7
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 14.142135620117188], num_nbr [1.0, 286.6449275362319]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 7.778174579143524], num_nbr [1.0, 92.65942028985508]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 4.5961940586566925], num_nbr [1.0, 35.833333333333336]
Calculateing adj matrix using xy only...
recommended radius =  3.0052037984132767 num_nbr=12.681159420289855
3.0052037984132767
radius= 3.0052037984132767 average number of neighbors for each spot is 12.681159420289855
 Cluster 7 has neighbors:
Dmain  12 :  137
Dmain  6 :  59
[12, 6]
SVGs for domain  7 : ['Pcp4', 'Snrpn', 'Gabbr1', 'Map1b', 'Sncb', 'Trnp1', 'Adarb1', 'Tcf25', 'Elmo1', 'Vamp2', 'Grina', 'Slc25a5', 'Ywhag', 'Tuba4a', 'Ncdn', 'Ndrg4', 'Adgrb1', 'Atp2a2', '2900097C17Rik', 'Cntn1', 'Amotl1', 'Atp6ap2', 'Nptn', 

In [54]:
target=8
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 14.142135620117188], num_nbr [1.0, 201.46875]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 7.778174579143524], num_nbr [1.0, 67.53125]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 4.5961940586566925], num_nbr [1.0, 26.96875]
Calculateing adj matrix using xy only...
Run 4: radius [3.0052037984132767, 4.5961940586566925], num_nbr [9.96875, 26.96875]
Calculateing adj matrix using xy only...
Run 5: radius [3.0052037984132767, 3.8006989285349846], num_nbr [9.96875, 15.8125]
Calculateing adj matrix using xy only...
Run 6: radius [3.0052037984132767, 3.4029513634741306], num_nbr [9.96875, 15.8125]
Calculateing adj matrix using xy only...
Run 7: radius [3.0052037984132767, 3.2040775809437037], num_nbr [9.96875, 15.8125]
Calculateing adj matrix using xy only...
Run 8: radius [3.

Run 75: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.96875, 15.8125]
Calculateing adj matrix using xy only...
Run 76: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.96875, 15.8125]
Calculateing adj matrix using xy only...
Run 77: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.96875, 15.8125]
Calculateing adj matrix using xy only...
Run 78: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.96875, 15.8125]
Calculateing adj matrix using xy only...
Run 79: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.96875, 15.8125]
Calculateing adj matrix using xy only...
Run 80: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.96875, 15.8125]
Calculateing adj matrix using xy only...
Run 81: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.96875, 15.8125]
Calculateing adj matrix using xy only...
Run 82: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.96875, 15.8125]
Calculateing adj matrix using xy only...
Run 83: 

TypeError: unsupported operand type(s) for ** or pow(): 'NoneType' and 'int'

In [55]:
target=9
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 14.142135620117188], num_nbr [1.0, 232.25]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 7.778174579143524], num_nbr [1.0, 82.1640625]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 4.5961940586566925], num_nbr [1.0, 33.359375]
Calculateing adj matrix using xy only...
recommended radius =  3.0052037984132767 num_nbr=12.2109375
3.0052037984132767
radius= 3.0052037984132767 average number of neighbors for each spot is 12.2109375
 Cluster 9 has neighbors:
Dmain  0 :  179
Dmain  12 :  147
[0, 12]
SVGs for domain  9 : ['Rims3', 'Stmn2', 'Map1b', 'Bex2', 'Slc17a6', 'Bex3', 'Gap43', 'Ldha', 'Ddah1', '6330403K07Rik', 'Tcf7l2', 'Tagln3', 'Tubb5', 'Nap1l5', 'Tubb2a', 'Cnih2', 'Dclk1', 'Rcn2', 'Fxyd7', 'Elavl2', 'Cbln1', 'Hlf', 'Oxr1', 'Wdr6', 'Rgs4', 'Acot13', 'Marcksl1', 'Timp3', '

In [56]:
target=10
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 14.142135620117188], num_nbr [1.0, 253.46067415730337]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 7.778174579143524], num_nbr [1.0, 89.59550561797752]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 4.5961940586566925], num_nbr [1.0, 35.438202247191015]
Calculateing adj matrix using xy only...
recommended radius =  3.0052037984132767 num_nbr=12.702247191011235
3.0052037984132767
radius= 3.0052037984132767 average number of neighbors for each spot is 12.702247191011235
 Cluster 10 has neighbors:
Dmain  16 :  117
Dmain  15 :  97
Dmain  14 :  76
Dmain  13 :  45
[16, 15, 14, 13]
SVGs for domain  10 : ['Hba-a1', 'Hbb-bt', 'Hba-a2', 'Nptxr', 'Hpcal1', 'Lypd1', 'Lmo3', 'Nr2f2', 'C1ql3']


In [57]:
target=11
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 14.142135620117188], num_nbr [1.0, 258.1212121212121]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 7.778174579143524], num_nbr [1.0, 85.15151515151516]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 4.5961940586566925], num_nbr [1.0, 33.34090909090909]
Calculateing adj matrix using xy only...
recommended radius =  3.0052037984132767 num_nbr=12.015151515151516
3.0052037984132767
radius= 3.0052037984132767 average number of neighbors for each spot is 12.015151515151516
 Cluster 11 has neighbors:
Dmain  1 :  244
Dmain  5 :  225
Dmain  13 :  103
Dmain  6 :  96
[1, 5, 13, 6]
SVGs for domain  11 : ['Mobp', 'Cldn11', 'Apod', 'Slc48a1', 'S100a16', 'Fabp5', 'Sirt2', 'Tmem88b', 'Prdx1', 'Josd2', 'Prdx6', 'Abca2', 'Kif5b', 'Stmn4', 'Gpm6b', 'Kif1b', 'Fez1', 'Tmbim6', 'Lamp1', 'Malat

In [58]:
target=12
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 14.142135620117188], num_nbr [1.0, 279.4065040650407]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 7.778174579143524], num_nbr [1.0, 92.95934959349593]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 4.5961940586566925], num_nbr [1.0, 36.073170731707314]
Calculateing adj matrix using xy only...
recommended radius =  3.0052037984132767 num_nbr=12.8130081300813
3.0052037984132767
radius= 3.0052037984132767 average number of neighbors for each spot is 12.8130081300813
 Cluster 12 has neighbors:
Dmain  9 :  147
Dmain  7 :  137
[9, 7]
SVGs for domain  12 : ['Prkcd', 'Dlgap3', 'Cck', 'Rora', 'Adarb1', 'Rasgrp1', 'Atp2b1', 'Ppp1r9b', 'Ramp3', 'Pcp4l1', 'Zic1', 'Nrn1', 'Nrxn3', 'Pakap-1', 'Necab2', 'Ptpn3', 'Rph3a', 'Lrrtm1', 'Stum', 'Cdkl5', 'Calb1', 'Tiam1', 'Shank3', 'B230334C0

In [59]:
target=13
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 14.142135620117188], num_nbr [1.0, 297.5]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 7.778174579143524], num_nbr [1.0, 95.40697674418605]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 4.5961940586566925], num_nbr [1.0, 36.604651162790695]
Calculateing adj matrix using xy only...
recommended radius =  3.0052037984132767 num_nbr=12.918604651162791
3.0052037984132767
radius= 3.0052037984132767 average number of neighbors for each spot is 12.918604651162791
 Cluster 13 has neighbors:
Dmain  11 :  103
[11]
SVGs for domain  13 : ['Nsg2', 'Gabarapl1', 'Suclg1', 'Slc25a5', 'Ahi1', 'Ndrg4', '1500011B03Rik', 'Ldhb', 'Hpcal4', 'Nnat', 'Ndufab1', 'Pld3', 'Pnmal2', 'Atp6ap2', 'Nsg1', 'Zwint', 'Hras', 'Rtn1', 'Atp5mpl', 'Pgam1', 'Abhd8', 'Tmem132a', '6330403K07Rik', 'Lancl1', 'Uqcrb

In [60]:
target=14
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 14.142135620117188], num_nbr [1.0, 274.27868852459017]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 7.778174579143524], num_nbr [1.0, 95.62295081967213]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 4.5961940586566925], num_nbr [1.0, 36.885245901639344]
Calculateing adj matrix using xy only...
recommended radius =  3.0052037984132767 num_nbr=13.0
3.0052037984132767
radius= 3.0052037984132767 average number of neighbors for each spot is 13.0
 Cluster 14 has neighbors:
No neighbor domain found, try bigger radius or smaller ratio.
None


TypeError: 'NoneType' object is not subscriptable

In [61]:
target=15
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 14.142135620117188], num_nbr [1.0, 249.1237113402062]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 7.778174579143524], num_nbr [1.0, 86.4020618556701]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 4.5961940586566925], num_nbr [1.0, 34.36082474226804]
Calculateing adj matrix using xy only...
recommended radius =  3.0052037984132767 num_nbr=12.381443298969073
3.0052037984132767
radius= 3.0052037984132767 average number of neighbors for each spot is 12.381443298969073
 Cluster 15 has neighbors:
Dmain  10 :  97
Dmain  8 :  80
[10, 8]
SVGs for domain  15 : ['6330403K07Rik', 'Rtl8a', 'Gap43', 'Pmch', 'Arxes2', 'Peg3', 'Impact', 'Naca', 'Agt', 'Bex1', 'Tceal3', 'Zcchc18', 'Ahi1', 'Bex3', 'Rnd2', 'Rtl8b', 'Tppp3', 'Gad2', 'Nsg1', 'Nrsn2', 'Tmem130', 'Resp18', 'Cartpt', 'Nap1l5',

In [62]:
target=16
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 14.142135620117188], num_nbr [1.0, 170.57894736842104]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 7.778174579143524], num_nbr [1.0, 63.81578947368421]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 4.5961940586566925], num_nbr [1.0, 27.587719298245613]
Calculateing adj matrix using xy only...
recommended radius =  3.0052037984132767 num_nbr=10.719298245614034
3.0052037984132767
radius= 3.0052037984132767 average number of neighbors for each spot is 10.719298245614034
 Cluster 16 has neighbors:
Dmain  10 :  117
Dmain  14 :  45
Dmain  4 :  10
[10, 14, 4]
SVGs for domain  16 : ['Cplx2', 'Chgb', 'Arpp21', 'Oip5os1', 'Pkig', 'Nrxn3', 'Slc30a3', 'Cadm3', 'Nrn1', 'Dgkb', 'Dbpht2', 'Btbd3', 'Pitpnm2', 'Sowaha', 'Wfs1', 'Plxnd1', '2010300C02Rik', 'Cacna2d1', 'Kcnj4', 'Wipf3', 'H

In [63]:
target=17
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 14.142135620117188], num_nbr [1.0, 209.38285714285715]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 7.778174579143524], num_nbr [1.0, 79.01714285714286]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 4.5961940586566925], num_nbr [1.0, 32.29142857142857]
Calculateing adj matrix using xy only...
recommended radius =  3.0052037984132767 num_nbr=11.794285714285714
3.0052037984132767
radius= 3.0052037984132767 average number of neighbors for each spot is 11.794285714285714
 Cluster 17 has neighbors:
Dmain  0 :  200
Dmain  8 :  19
[0, 8]
SVGs for domain  17 : ['Resp18', 'Nap1l5', 'Tmem130', 'Pld3', 'Maged1', 'Dynll1', 'Tbca', 'Saraf', 'Fxyd6', 'Tma7', 'Erp29', 'Gm1673', 'Cbarp', 'Tmem59l', 'Ssr4', 'Gap43', 'Tppp3', 'Araf', 'Peg3', 'Zcchc18', 'Sub1', 'Gprasp2', 'Ldha', '6330403K